# Setup

In [1]:
import pandas as pd
import copy
import numpy as np
import warnings
import math

from pprint import pformat
import xlsxwriter
import os
import json
import pprint
from datetime import datetime

# Functions

## Import

<div class="alert alert-info">
    
Funktionen, die benötigt werden, um Input-Exceldatei auszulesen und die notwendigen excel-sheets als panda-dataframe im dictionary-Format zurückzugeben.

In [2]:
def read_excel_file(filepath, sheet_names):
    print("Apply strategies: read excel-file ...")
    xl = pd.ExcelFile(filepath)
    dfs = {sheet_name: xl.parse(sheet_name).T.to_dict() for sheet_name in sheet_names}
    
    return tuple(dfs.values())

In [4]:
def unzip_dict(df):
    df_new = [df[a] for a in df]
    return df_new

# Transformation & Calculation Functions

## D_PSE (,U_PSE)

In [5]:
def add_träger_column(df):
    print("Apply strategies - PP: add täger-column ...")
    for a in df:
        #lowest_e = True if a+1 in df and df[a+1]["Ebene"]<= df[a]["Ebene"] else False
        current_e=a["Ebene"]
        current_nr=a["Nr."]
        e0, e1, e2, e3, e4 = (
            current_nr if current_e == 0 else e0 if current_e >= 0 else "-", 
            current_nr if current_e == 1 else e1 if current_e >= 1 else "-",
            current_nr if current_e == 2 else e2 if current_e >= 2 else "-",
            current_nr if current_e == 3 else e3 if current_e >= 3 else "-",
            current_nr if current_e == 4 else "-"
        )
        upper_e = (
            e3 if current_e == 4
            else e2 if current_e == 3
            else e1 if current_e == 2
            else e0 if current_e == 1
            else "-"
        )
        a["Träger"]=upper_e
    
    return df

In [7]:
def transform_excel_sheet_to_matrix(df):
    print("Apply strategies - PP: transform excel-sheet to D_PSE matrix ...")
    df_new = []
    for a_count, PSE_a in enumerate(df):
        df_new.append({
            "name":PSE_a['Bezeichnung'],
            "reference product":PSE_a['Bezeichnung'],
            "unit":"pcs.",
            "code":PSE_a['Nr.'],
            "level":PSE_a['Ebene'],
            "exchanges":[{
                "name":PSE_a['Bezeichnung'],
                "reference product":PSE_a['Bezeichnung'],
                "unit":"pcs.",
                "code":PSE_a['Nr.'],
                "type":"production",
                "amount":1
            }]
        })
        for a2 in df:
            if a2['Träger']==df_new[a_count]["code"]:
                df_new[a_count]["exchanges"].append({
                    "name":a2['Bezeichnung'],
                    "reference product":a2['Bezeichnung'],
                    "unit":"pcs.",
                    "code":a2['Nr.'],
                    "type":"technosphere",
                    "amount":-a2['Menge']
                })
    
    return df_new

In [9]:
def sort_column_by_level(matrix):
    print("Apply strategies - PP: sort matrix-column by level (descending) ...")
    
    matrix_new=sorted(matrix, key=lambda d: d['level'])
    
    return matrix_new

## D_PPZ, U_PPZ

<div class="alert alert-info">
    
``read_excel_file(filepath, sheet_names)`` ausführen und "4-Prozesse"-Tabelle auswählen

In [10]:
def filter_out_PPZ_PZ_old(df):
    print("Apply strategies - PP: create '4.1' Tab.")
    df_PPZ=[]
    for c in df:
        if df[c]["Nr."].startswith("PP"):
            df_PPZ.append(df[c])
            
    return df_PPZ

In [11]:
def filter_out_PPZ_PZ(df):
    print("Apply strategies - PP: create '4.1' Tab.")
    df_PPZ = [df[c] for c in df if df[c]["Nr."].startswith("PP")]
    return df_PPZ

In [12]:
def transform_df_PPZ(df,D_PSE):
    print("Apply strategies - PP: transform '4.1' Tab.")
    for c in df:
        c['Träger']=[]
        code_list=[ v for k, v in c.items() if k.startswith('Träger ')]
        n_bearbeitet_list=[ v for k, v in c.items() if k.startswith('n ')]
        for i in range(len(code_list)):
            if not math.isnan(n_bearbeitet_list[i]):
                c['Träger'].append({
                    'code':code_list[i],
                    'n_bearbeitet':n_bearbeitet_list[i],
                    'n':1
                })
        for a in D_PSE:
            for exc in a['exchanges']:
                for t in c['Träger']:
                    if exc['type']=='technosphere' and exc['code'] == t['code']:
                        t['n']=-exc['amount']
        entries_to_remove=[ k for k, v in c.items() if (k.startswith('Träger ') or k.startswith('n '))]
        for k in entries_to_remove:
            c.pop(k, None)
        
    df_xPZ_v2=[]
    for PZ in df:
        PZ['ÖW']=[]
        if not any(d['Nr.'] == PZ['Nr.'] for d in df_xPZ_v2):
            df_xPZ_v2.append(PZ)
    for PZ1 in df:
        for PZ2 in df_xPZ_v2:
            if PZ1['Nr.']==PZ2['Nr.'] and PZ1['ÖK-Wert']!=0:
                PZ2['ÖW'].append({
                    'Nr.':PZ1['Ökologie-kriterium'],
                    'amount':PZ1['ÖK-Wert']
                })
    for PZ2 in df_xPZ_v2:
        del PZ2['Ökologie-kriterium']
        del PZ2['ÖK-Wert']
    
    return df_xPZ_v2

In [13]:
def create_AF(df,allocation_key):
    print("Apply strategies - PP: create AF")
    if allocation_key=='pcs.':
        for PZ in df:
            for PZT in PZ['Träger']:
                PZT['af'] = PZT['n']/sum(item['n_bearbeitet'] for item in PZ['Träger'])
                
    return df    

In [14]:
def add_AF_to_A(df_PPZ, D_PSE):
    print("Apply strategies - PP: add AF to '4.1' Tab.")
    for PSE in D_PSE:
        for PZ in df_PPZ:
            for PZT in PZ['Träger']:
                if PZT['code']==PSE['code']:
                    PSE['exchanges'].append({
                        "name":PZ['Prozess-schritt'],
                        "reference product":PZ['Prozess-schritt'],
                        "unit":"exc.",
                        "code":PZ['Nr.'],
                        "type":"technosphere",
                        "amount":-PZT['af']
                    })
                    
    return D_PSE

## lci_PP

In [15]:
def write_PP_lci(df_PPZ,D_PSE,df_ÖK):
    print("Apply strategies - PP: write lci_PP")
    for PPZ in df_PPZ:
        activity = {
            "name":PPZ['Prozess-schritt'],
            "reference product":PPZ['Prozess-schritt'],
            "unit":"exc.",
            "code":PPZ['Nr.'],
            "exchanges":[{
                "name":PPZ['Prozess-schritt'],
                "reference product":PPZ['Prozess-schritt'],
                "unit":"exc.",
                "code":PPZ['Nr.'],
                "type":"production",
                "amount":1
            }]
        }
        for öw in PPZ['ÖW']:
            activity['exchanges'].append({
                "code":öw['Nr.'],
                "type":"biosphere",
                "amount":-öw['amount']
            })
        D_PSE_v2.append(activity)

    for act in D_PSE:
        for exc in act['exchanges']:
            for ök in df_ÖK:
                if exc['code'] == ök['Nr.']:
                    exc['name']=ök['Kriterium']
                    exc['reference product']=ök['Kriterium']
                    exc['unit']=ök['Einheit']
                    if ök['Generisches Ökologiekriterium'].startswith("Emissionen"):
                        exc['amount']=exc['amount']*(-1)

    return D_PSE

# main.ipynb simulator

In [19]:
filepath = r"data/231105_LCA_v24_rsc_kf.xlsm"
sheet_names = [
    "1-Produktstruktur", 
    "2-Funktionsstruktur",
    "3-Ökologiekriterien",
    "4-Prozesse",
    "5-MTTFF MTBF",
    "6-Rückführungswahrs.",
    "7a-Transformbez_PF_gew",
    "7b-Transformbez_FP_gew",
    "8-Nutzungsprofile"
]

################################################################

df_PSE_v1 = unzip_dict(read_excel_file(filepath=filepath, sheet_names=sheet_names)[0]) #<--
df_PSE_v2 = add_träger_column(df=df_PSE_v1)
D_PSE_v1 = transform_excel_sheet_to_matrix_old(df=df_PSE_v2)
D_PSE_v2 = sort_column_by_level(matrix=D_PSE_v1)

df_PZ_ges = read_excel_file(filepath=filepath, sheet_names=sheet_names)[3] #<--
df_PPZ_v1 = filter_out_PPZ_PZ_old(df=df_PZ_ges)
df_PPZ_v2 = transform_df_PPZ(df=df_PPZ_v1,D_PSE=D_PSE_v2) 
df_PPZ_v3 = create_AF(df=df_PPZ_v2,allocation_key='pcs.') #<--

D_PSE_v3 = add_AF_to_A(df_PPZ=df_PPZ_v3, D_PSE=D_PSE_v2)

df_ÖK_v1 = unzip_dict(read_excel_file(filepath=filepath, sheet_names=sheet_names)[2])

lci_PP = write_PP_lci(df_PPZ=df_PPZ_v3, D_PSE=D_PSE_v2, df_ÖK=df_ÖK_v1)

Apply strategies: read excel-file ...
Apply strategies - PP: add täger-column ...
Apply strategies - PP: transform excel-sheet to D_PSE matrix ...
Apply strategies - PP: sort matrix-column by level (descending) ...
Apply strategies: read excel-file ...
Apply strategies - PP: create '4.1' Tab.
Apply strategies - PP: transform '4.1' Tab.
Apply strategies - PP: create AF
Apply strategies - PP: add AF to '4.1' Tab.
Apply strategies: read excel-file ...
Apply strategies - PP: write lci_PP


In [20]:
lci_PP

[{'name': 'Stanzmaschine',
  'reference product': 'Stanzmaschine',
  'unit': 'pcs.',
  'code': 'M201830',
  'level': 0.0,
  'exchanges': [{'name': 'Stanzmaschine',
    'reference product': 'Stanzmaschine',
    'unit': 'pcs.',
    'code': 'M201830',
    'type': 'production',
    'amount': 1},
   {'name': 'Grundgestell',
    'reference product': 'Grundgestell',
    'unit': 'pcs.',
    'code': 'M215720',
    'type': 'technosphere',
    'amount': -1.0},
   {'name': 'Handlingsystem',
    'reference product': 'Handlingsystem',
    'unit': 'pcs.',
    'code': 'M216421',
    'type': 'technosphere',
    'amount': -1.0},
   {'name': 'Antriebssystem',
    'reference product': 'Antriebssystem',
    'unit': 'pcs.',
    'code': 'M217221',
    'type': 'technosphere',
    'amount': -1.0},
   {'name': 'Werkzeugwechselsystem',
    'reference product': 'Werkzeugwechselsystem',
    'unit': 'pcs.',
    'code': 'M218130',
    'type': 'technosphere',
    'amount': -1.0},
   {'name': 'Werkzeugsystem',
    're

# Functions - old

In [6]:
def add_träger_column_old(df):
    print("Apply strategies - PP: add Träger-column ...")

    for a in df:
        current_e = int(a["Ebene"]) if not math.isnan(a["Ebene"]) else 0
        current_nr = a["Nr."]
        
        e_values = [current_nr if current_e >= i else "-" for i in range(5)]
        upper_e = e_values[current_e - 1] if current_e > 0 else "-"

        a["Träger"] = upper_e

    return df

In [8]:
def transform_excel_sheet_to_matrix_old(df):
    print("Apply strategies - PP: transform excel-sheet to D_PSE matrix ...")

    df_new = [{
        "name": PSE['Bezeichnung'],
        "reference product": PSE['Bezeichnung'],
        "unit": "pcs.",
        "code": PSE['Nr.'],
        "level": PSE['Ebene'],
        "exchanges": [{
            "name": PSE['Bezeichnung'],
            "reference product": PSE['Bezeichnung'],
            "unit": "pcs.",
            "code": PSE['Nr.'],
            "type": "production",
            "amount": 1
        }]}
        for PSE in df]

    for a2 in df:
        for a_count, PSE in enumerate(df_new):
            if a2['Träger'] == PSE["code"]:
                PSE["exchanges"].append({
                    "name": a2['Bezeichnung'],
                    "reference product": a2['Bezeichnung'],
                    "unit": "pcs.",
                    "code": a2['Nr.'],
                    "type": "technosphere",
                    "amount": -a2['Menge']
                })

    return df_new

# Spielwiese

In [ ]:
D_PSE_v2

In [ ]:
df_PPZ_v2=[]
for PPZ in df_PPZ_v1:
    PPZ['ÖW']=[]
    if not any(d['Nr.'] == PPZ['Nr.'] for d in df_PPZ_v2):
        df_PPZ_v2.append(PPZ)
        
for PPZ1 in df_PPZ_v1:
    for PPZ2 in df_PPZ_v2:
        if PPZ1['Nr.']==PPZ2['Nr.'] and PPZ1['ÖK-Wert']!=0:
            PPZ2['ÖW'].append({'Nr.':PPZ1['Ökologie-kriterium'],
                               'Amount':PPZ1['ÖK-Wert']})
for PPZ2 in df_PPZ_v2:
    del PPZ2['Ökologie-kriterium']
    del PPZ2['ÖK-Wert']

        
df_PPZ_v2

In [ ]:
for c in df_PPZ_v1:
    c['Träger']=[ v for k, v in c.items() if k.startswith('Träger ')]
    c['n_bearbeitet']=[ v for k, v in c.items() if k.startswith('n ')]
    c['n']=[]
    for a in D_PSE_v2 :
        for exc in a['exchanges']:
            for t in c['Träger']:
                if exc['type']=='technosphere' and exc['code'] == t:
                    c['n'].append(-exc['amount'])
                else:
                    
    
    
    entries_to_remove=[ k for k, v in c.items() if (k.startswith('Träger ') or k.startswith('n '))]
    for k in entries_to_remove:
        c.pop(k, None)


In [ ]:
df_PPZ=[]
for c in df:
    if df[c]["Nr."].startswith("PP"):
        df_PPZ.append(df[c])
df_PPZ

In [ ]:
df_D_PSE_v3 = []

for a in df_D_PSE_v2:
    df_D_PSE_v3.append({
        "name":df_D_PSE_v2[a]['Bezeichnung'],
        "reference product":df_D_PSE_v2[a]['Bezeichnung'],
        "unit":"pcs.",
        "code":df_D_PSE_v2[a]['Nr.'],
        "level":df_D_PSE_v2[a]['Ebene'],
        "exchanges":[{
            "name":df_D_PSE_v2[a]['Bezeichnung'],
            "reference product":df_D_PSE_v2[a]['Bezeichnung'],
            "unit":"pcs.",
            "type":"production",
            "amount":1
        }]
    })
    for a2 in df_D_PSE_v2:
        if df_D_PSE_v2[a2]['Träger']==df_D_PSE_v3[a]["code"]:
            df_D_PSE_v3[a]["exchanges"].append({
                "name":df_D_PSE_v2[a2]['Bezeichnung'],
                "reference product":df_D_PSE_v2[a2]['Bezeichnung'],
                "unit":"pcs.",
                "type":"technosphere",
                "amount":-df_D_PSE_v2[a2]['Menge']
            })
        
    
    

    
    
df_D_PSE_v3